# Busca sem informação

## Modelagem do problema

In [ ]:
from __future__ import annotations
from dataclasses import dataclass

from typing import List
from typing import Callable


@dataclass
class Estado:
    nodo: List
    pai: Estado
    acao: str
    custo: float

    def __eq__(self, outro):
        if not isinstance(outro, Estado):
            return False

        return self.nodo == outro.nodo

    def __hash__(self):
        return hash(frozenset(self.nodo))   

@dataclass
class Problema:
    estado_inicial: Estado
    acoes: List[str]
    funcao_sucessora: Callable[[Estado], List[Estado]]
    funcao_objetivo: Callable[[Estado], bool] 

    def solucao(self, estado: Estado) -> List[Estado]:
        lista = [estado]

        while estado.pai != None:
            estado = estado.pai
            lista.append(estado)

        lista.reverse()
        
        return lista



In [ ]:
from pprint import pprint

def busca(problema: Problema) -> List:
    """
    Implementação do algoritmo de busca.

    :param problema: definição do problema
    """

    # Inicialmente somente o estado inicial está na fronteira
    fronteira = [problema.estado_inicial]

    while True:

        pprint('fronteira: ')
        pprint([e.nodo for e in fronteira])
        print()

        # Verifica se a fronteira está vazia
        if not fronteira:
            raise RuntimeError('Nenhuma solução encontrada')

        # Remove o próximo elemento da fronteira
        estado = fronteira.pop(0)     # FIFO - busca em largura
        #estado = fronteira.pop()      # LIFO - busca em profundidade

        #fronteira.sort(key=lambda e: e.custo)
        #estado = fronteira.pop(0)     # HEAP - busca de custo uniforme

        pprint('Estado visitado: ')
        pprint(estado.nodo)
        pprint(f'Custo do caminho: {estado.custo}')
        print()

        # Testa o objetivo
        if problema.funcao_objetivo(estado):
            return problema.solucao(estado)

        # Expande o estado atual e adiciona os vizinhos na fronteira
        fronteira += problema.funcao_sucessora(estado) 



## Problema 1: Torre de Hanói

In [ ]:
"""
# Modelagem do problema

Estado:
    - Cada estado representa cada uma das torres como uma lista
    - Cada disco é representado por um valor inteiro indicando o tamanho
      do disco 

          A       B    C
    [ [3, 2, 1], [ ], [ ] ]

          1
         2 2
        3 3 3
"""

acoes = {
    'A -> B': (0, 1), 
    'A -> C': (0, 2), 
    'B -> A': (1, 0), 
    'B -> C': (1, 2),
    'C -> A': (2, 0), 
    'C -> B': (2, 1)
}

visitado = []

def expansao(estado: Estado) -> List[Estado]:
    vizinhanca = []
    nodo = estado.nodo

    def copia_torres():
        return [nodo[0].copy(), nodo[1].copy(), nodo[2].copy()]

    for acao, movimento in acoes.items():
        de, para = movimento

        if nodo[de] and (not nodo[para] or nodo[de][-1] < nodo[para][-1]):
            torres = copia_torres()
            torres[para].append(torres[de].pop())
            
            novo_estado = Estado(
                nodo = torres,
                pai = estado,
                acao = acao,
                custo = estado.custo + 1,
            )

            # Expande o no somente se ele ainda não foi visitado
            if novo_estado not in visitado:
                vizinhanca.append(novo_estado)
                visitado.append(novo_estado)

    return vizinhanca


def teste_objetivo(estado: Estado):
    return estado.nodo == [[], [], [3, 2, 1]]


problema = Problema(
    estado_inicial = Estado(
                        nodo = [[3, 2, 1], [], []],
                        pai = None,
                        custo = 0,
                        acao = None,
                    ),
    acoes=['A -> B', 'A -> C', 
           'B -> A', 'B -> C',
           'C -> A', 'C -> B'],
    funcao_sucessora=expansao,
    funcao_objetivo=teste_objetivo,
)

solucao = busca(problema)

'fronteira: '
[[[3, 2, 1], [], []]]

'Estado visitado: '
[[3, 2, 1], [], []]
'Custo do caminho: 0'

'fronteira: '
[[[3, 2], [1], []], [[3, 2], [], [1]]]

'Estado visitado: '
[[3, 2], [1], []]
'Custo do caminho: 1'

'fronteira: '
[[[3, 2], [], [1]], [[3], [1], [2]], [[3, 2, 1], [], []]]

'Estado visitado: '
[[3, 2], [], [1]]
'Custo do caminho: 1'

'fronteira: '
[[[3], [1], [2]], [[3, 2, 1], [], []], [[3], [2], [1]]]

'Estado visitado: '
[[3], [1], [2]]
'Custo do caminho: 2'

'fronteira: '
[[[3, 2, 1], [], []], [[3], [2], [1]], [[3, 1], [], [2]], [[3], [], [2, 1]]]

'Estado visitado: '
[[3, 2, 1], [], []]
'Custo do caminho: 2'

'fronteira: '
[[[3], [2], [1]], [[3, 1], [], [2]], [[3], [], [2, 1]]]

'Estado visitado: '
[[3], [2], [1]]
'Custo do caminho: 2'

'fronteira: '
[[[3, 1], [], [2]], [[3], [], [2, 1]], [[3, 1], [2], []], [[3], [2, 1], []]]

'Estado visitado: '
[[3, 1], [], [2]]
'Custo do caminho: 3'

'fronteira: '
[[[3], [], [2, 1]], [[3, 1], [2], []], [[3], [2, 1], []]]

'Estado vi

In [ ]:
for estado in solucao:
    print(f'{str(estado.acao):10} {str(estado.nodo[0]):10s} {str(estado.nodo[1]):10s} {str(estado.nodo[2]):10s} - {estado.custo}')


None       [3, 2, 1]  []         []         - 0
A -> C     [3, 2]     []         [1]        - 1
A -> B     [3]        [2]        [1]        - 2
C -> B     [3]        [2, 1]     []         - 3
A -> C     []         [2, 1]     [3]        - 4
B -> A     [1]        [2]        [3]        - 5
B -> C     [1]        []         [3, 2]     - 6
A -> C     []         []         [3, 2, 1]  - 7
